In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
from torch.utils.data import Dataset

# Tải mô hình và tokenizer

In [2]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

c:\Users\lyhie\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Gán token padding là eos_token

In [3]:
tokenizer.pad_token = tokenizer.eos_token

# Tải dữ liệu từ Huggingface dataset

In [4]:
dataset = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")

# Lấy ra một phần dữ liệu từ tập train

In [5]:
dataset = dataset['train'].select(range(1000))

# Tiền xử lý dữ liệu: mã hóa các văn bản

In [6]:
def preprocess_data(examples):
    # Mã hóa cả input và output, gắn chặt input và output thành một chuỗi duy nhất
    inputs = examples['input']
    outputs = examples['output']
    
    # Gắn input và output lại với nhau
    input_output = [inp + tokenizer.eos_token + out for inp, out in zip(inputs, outputs)]
    
    # Mã hóa chuỗi kết hợp input-output
    encodings = tokenizer(input_output, truncation=True, padding="max_length", max_length=512)
    
    # Trả về giá trị mã hóa
    return encodings

# Áp dụng tiền xử lý dữ liệu

In [7]:
tokenized_datasets = dataset.map(preprocess_data, batched=True)

# Chia tập dữ liệu thành training và validation

In [8]:
train_val_split = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = train_val_split['train']
val_dataset = train_val_split['test']

KeyError: "Column train not in the dataset. Current columns in the dataset: ['instruction', 'input', 'output', 'input_ids', 'attention_mask']"

# Chuyển đổi thành Dataset cho PyTorch

In [8]:
class HealthDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        # Trả về các input_ids, attention_mask, và labels
        item = {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),  
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.encodings['input_ids'][idx])  # Sử dụng input_ids làm labels
        }
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Tạo Dataset cho Trainer

In [9]:
train_dataset = HealthDataset(train_dataset)
val_dataset = HealthDataset(val_dataset)

# Định nghĩa tham số huấn luyện

In [10]:
training_args = TrainingArguments(
    output_dir='./results',              # Nơi lưu kết quả
    num_train_epochs=3,                  # Số epoch huấn luyện
    per_device_train_batch_size=4,       # Batch size cho train
    per_device_eval_batch_size=4,        # Batch size cho validation
    warmup_steps=500,                    # Số bước khởi động để tăng dần learning rate
    weight_decay=0.01,                   # Trọng số cho việc giảm weight
    logging_dir='./logs',                # Thư mục lưu log
)

# Huấn luyện mô hình

In [11]:
trainer = Trainer(
    model=model,                         # Mô hình GPT-2
    args=training_args,                  # Tham số huấn luyện
    train_dataset=train_dataset,         # Dữ liệu huấn luyện
    eval_dataset=val_dataset             # Dữ liệu kiểm tra (validation)
)

trainer.train()

  0%|          | 0/75711 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Lưu mô hình và tokenizer

In [ ]:
model.save_pretrained('./health_chatbot_model')
tokenizer.save_pretrained('./health_chatbot_model')